In [1]:
# Import all the libraries
import pandas as pd
import glob
import os
import re
from pprint import pprint

In [2]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\SD\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['general'])
]

# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['primary'])
]


In [3]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\SD\20161108__sd__general__precinct.csv


In [4]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\SD\20160607__sd__primary__aurora__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\SD\20160607__sd__primary__brule__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\SD\20160607__sd__primary__clark__precinct.csv
C:\Huy Phan\College\VoterTurnout\data\SD\20160607__sd__primary__clay__precinct.csv


In [6]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
            
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
            df = df[~df['precinct'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]   

        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]               
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
pri_combined_df

,county,precinct,office,district,party,candidate,votes
0,Aurora,AURORA1,President,NaN,Democratic,Bernie Sanders,12
1,Aurora,AURORA2,President,NaN,Democratic,Bernie Sanders,6
2,Aurora,AURORA3,President,NaN,Democratic,Bernie Sanders,57
3,Aurora,AURORA5,President,NaN,Democratic,Bernie Sanders,11
4,Aurora,AURORA7,President,NaN,Democratic,Bernie Sanders,46
...,...,...,...,...,...,...,...
145,Clay,CLAY1906 VERMILLION CENTRAL,President,NaN,REP,John R. Kasich,18
146,Clay,CLAY1906 VERMILLION CENTRAL,President,NaN,REP,Ted Cruz,7
147,Clay,CLAY1906 VERMILLION CENTRAL,President,NaN,REP,Donald J. Trump,25
148,Clay,CLAY1906 VERMILLION CENTRAL,President,NaN,DEM,Bernie Sanders,119


In [29]:
# Checking any suspicious precinct, county
suspicious_precincts = pri_combined_df[
    pri_combined_df["precinct"].isna() |
    pri_combined_df["precinct"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["precinct"].str.strip().str.upper().isin(["NONE", "NAN"]) |
    pri_combined_df["county"].isna() |
    pri_combined_df["county"].str.strip().str.upper().str.contains(r"\w*TOTAL\w*", na=False) |
    pri_combined_df["county"].str.strip().str.upper().isin(["NONE", "NAN"])
]

In [7]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]

primary_data

,precinct,party,candidate,votes
0,AURORA1,Democratic,Bernie Sanders,12
1,AURORA2,Democratic,Bernie Sanders,6
2,AURORA3,Democratic,Bernie Sanders,57
3,AURORA5,Democratic,Bernie Sanders,11
4,AURORA7,Democratic,Bernie Sanders,46
...,...,...,...,...
145,CLAY1906 VERMILLION CENTRAL,REP,John R. Kasich,18
146,CLAY1906 VERMILLION CENTRAL,REP,Ted Cruz,7
147,CLAY1906 VERMILLION CENTRAL,REP,Donald J. Trump,25
148,CLAY1906 VERMILLION CENTRAL,DEM,Bernie Sanders,119


In [8]:
# Viewing candidate data
primary_data["candidate"].value_counts(dropna=False)

candidate
Bernie Sanders     30
Hillary Clinton    30
John R. Kasich     30
Ted Cruz           30
Donald J. Trump    30
Name: count, dtype: int64

In [9]:
# Cleaning Candidates

# Turning all primary data to uppercase
primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()

unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"\w*ATTERING\w*",
    r"\w*UNINSTRUCTED\w*",
    r"UNCOMMITTED", r"OTHER",r"Total Votes Cast",r"No Preference"
]

pattern = "|".join(unwanted_keywords)

# Assuming candidate column is already string and uppercase
primary_data = primary_data[~primary_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

#fIXING DE LA FUENTE
primary_data.loc[
    (
        primary_data["candidate"].str.contains("rocky.*fuente", case=False, na=False) |
        primary_data["candidate"].str.contains("LA FUENTE", case=False, na=False) |
        primary_data["candidate"].str.contains("\w*Fuque\w*", case=False, na=False)
    ),
    "candidate"
] = "LA FUENTE"

# # Fixing williams
primary_data.loc[
    primary_data["candidate"].str.contains("WILLIAMS", case=False, na=False),
    "candidate"
] = "WILLIAMS"

primary_data.loc[:, "candidate"] = primary_data["candidate"].replace({
    "Donald I. Trump": "Donald J. Trump",
})

# # Fixing the , candidate
# primary_data["candidate"] = (
#     primary_data["candidate"].str.split(",")
#     .str[0]
#     .str.strip()
#     )


# #Selecting only last name
primary_data["candidate"] = primary_data["candidate"].str.split().str[-1]

# # Fixing Christie
# primary_data.loc[
#     primary_data["candidate"].str.contains("CHRISTLE", case=False, na=False),
#     "candidate"
# ] = "CHRISTIE"

# # Fixing KASICH
# primary_data.loc[
#     primary_data["candidate"].str.contains("KAISCH", case=False, na=False),
#     "candidate"
# ] = "CHRISTIE"

# # Fixing O'MALLEY
# primary_data.loc[
#     primary_data["candidate"].str.contains("O'MALLEY", case=False, na=False),
#     "candidate"
# ] = "O'MAILEY"

# # Fixing O'MALLEY
# primary_data.loc[
#     primary_data["candidate"].str.contains("PATAKL", case=False, na=False),
#     "candidate"
# ] = "PATAKI"

# # Fixing O'MALLEY
# primary_data.loc[
#     primary_data["candidate"].str.contains("RUBLO", case=False, na=False),
#     "candidate"
# ] = "RUBIO"

# # Fixing Huckabee
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ABEE\w*", case=False, na=False),
#     "candidate"
# ] = "HUCKABEE"

# # Fixing Fiorina
# primary_data.loc[
#     primary_data["candidate"].str.contains("\w*ORINA\W*", case=False, na=False),
#     "candidate"
# ] = "FIORINA"

# # Fixing Gray
# primary_data.loc[
#     primary_data["candidate"].str.contains("GREY", case=False, na=False),
#     "candidate"
# ] = "GRAY"

# # Fixing Wilson
# primary_data.loc[
#     primary_data["candidate"].str.contains("WISON", case=False, na=False),
#     "candidate"
# ] = "WILSON"


# # Fixing separator 
# primary_data["candidate"] = (
#     primary_data["candidate"]
#     .str.split(r"\s*(?:and|/|&|–|-|\+)\s*", n=1, expand=True)[0]
#     .str.strip()
#     .str.upper()
# )

# # Fixing McMullin

# primary_data.loc[
#     (
#         primary_data["candidate"].str.contains("MCMULLIN", case=False, na=False) |
#         primary_data["candidate"].str.contains("EVAN MCMULLEN", case=False, na=False)
#     ),
#     "candidate"
# ] = "MCMULLIN"

# primary_data.loc[
#     primary_data["candidate"].str.contains("De La Fuen", case=False, na=False),
#     "candidate"
# ] = "FUENTE D"

# primary_data.loc[:,"candidate"] = (
#     primary_data["candidate"].str.split().str[0].str.upper()
# )

primary_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_36512\1590360723.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  primary_data["candidate"] = primary_data["candidate"].astype(str).str.upper()


candidate
SANDERS    30
CLINTON    30
KASICH     30
CRUZ       30
TRUMP      30
Name: count, dtype: int64

In [10]:
# Viewing Party
primary_data["party"].value_counts(dropna=False)

party
REP           60
DEM           40
Republican    30
Democratic    20
Name: count, dtype: int64

In [11]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown

#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )

primary_data["party"].value_counts(dropna=False)


party
REP           60
DEM           40
Republican    30
Democratic    20
Name: count, dtype: int64

In [12]:
# Cleaning Party
# Turning all general data party to uppercase
primary_data["party"] = primary_data["party"].astype(str).str.upper()

# primary_data["party"] = primary_data.apply(
#     lambda row: fill_party_from_data(row, primary_data),
#     axis=1
# )
primary_data["party"] = (
    primary_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GREEN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LBT": "LIB",
        "L": "LIB",
        "CONSTITUTION": "CON",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK"
    })
)
primary_data["party"].value_counts(dropna=False)

party
REP    90
DEM    60
Name: count, dtype: int64

In [13]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"]
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_CRUZ,pri_rep_KASICH,pri_rep_TRUMP
0,AURORA1,30,12,18,8,42
1,AURORA2,8,6,12,5,12
2,AURORA3,71,57,13,5,42
3,AURORA5,12,11,2,5,11
4,AURORA7,29,46,5,4,29
5,BRULE1,103,118,34,44,152
6,BRULE2,46,49,8,5,23
7,BRULE3,26,35,15,6,38
8,BRULE4,3,3,2,1,6
9,BRULE5,4,14,2,2,26


In [15]:
# Process general files
general_df_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)
        
        # Combine precinct as county + precinct
        df["precinct"] = df["county"].astype(str) + df["precinct"].astype(str)
        df["precinct"] = df["precinct"].str.upper()
        df["county"] = df["county"].str.upper()

        # Cleaning maybe total precinct
        if 'precinct' in df.columns:
            df['precinct'] = df['precinct'].astype(str)                  
            df = df[df['precinct'].str.upper() != "TOTAL"]               
            df = df[df['precinct'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['precinct'].str.upper() != "NAN"]                
        
        if 'county' in df.columns:
            df['county'] = df['county'].astype(str)                  
            df = df[df['county'].str.upper() != "TOTAL"]               
            df = df[df['county'].str.strip().str.upper() != "TOTAL"]   
            df = df[df['county'].str.upper() != "NAN"]       
            df = df[~df['county'].str.upper().str.contains(r"\w*TOTAL\w*", na=False)]      
            
        # Filtering out only President 
        if 'office' in df.columns:
            df = df[df["office"] == "President"]
        
        # Dropping duplicates value
        df = df.drop_duplicates()
        
        general_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(general_df_list, ignore_index=True)
# Checking any suspicious precinct
suspicious_precincts = gen_combined_df[
    gen_combined_df["precinct"].str.strip().str.upper().isin(["TOTAL", "NAN"])
]
gen_combined_df


,county,precinct,office,district,candidate,party,votes
0,AURORA,AURORA1,President,NaN,Donald J. Trump,REP,230.0
1,AURORA,AURORA2,President,NaN,Donald J. Trump,REP,120.0
2,AURORA,AURORA3,President,NaN,Donald J. Trump,REP,290.0
3,AURORA,AURORA5,President,NaN,Donald J. Trump,REP,128.0
4,AURORA,AURORA7,President,NaN,Donald J. Trump,REP,206.0
...,...,...,...,...,...,...,...
3068,ZIEBACH,ZIEBACHPRECINCT-3 DAVIS,President,NaN,Darrell L. Castle,CON,5.0
3069,ZIEBACH,ZIEBACHPRECINCT-4 GLAD VALLEY,President,NaN,Darrell L. Castle,CON,0.0
3070,ZIEBACH,ZIEBACHPRECINCT-6 REDELM (PRECINCT-5 NORTH DUP...,President,NaN,Darrell L. Castle,CON,10.0
3071,ZIEBACH,ZIEBACHPRECINCT-7 RED SCAFFOLD,President,NaN,Darrell L. Castle,CON,3.0


In [16]:
# Select only the relevant columns
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]

general_data

,precinct,party,candidate,votes
0,AURORA1,REP,Donald J. Trump,230.0
1,AURORA2,REP,Donald J. Trump,120.0
2,AURORA3,REP,Donald J. Trump,290.0
3,AURORA5,REP,Donald J. Trump,128.0
4,AURORA7,REP,Donald J. Trump,206.0
...,...,...,...,...
3068,ZIEBACHPRECINCT-3 DAVIS,CON,Darrell L. Castle,5.0
3069,ZIEBACHPRECINCT-4 GLAD VALLEY,CON,Darrell L. Castle,0.0
3070,ZIEBACHPRECINCT-6 REDELM (PRECINCT-5 NORTH DUP...,CON,Darrell L. Castle,10.0
3071,ZIEBACHPRECINCT-7 RED SCAFFOLD,CON,Darrell L. Castle,3.0


In [17]:
# Viewing candidate data
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()
general_data["candidate"].value_counts(dropna=False)

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_36512\1743146100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


candidate
GARY JOHNSON          716
HILLARY CLINTON       716
DONALD J. TRUMP       644
DARRELL L. CASTLE     644
DONALD TRUMP           72
DARRELL CASTLE         72
TRUMP & PENCE          51
JOHNSON & WELD         30
CLINTON & KAINE        30
CASTLE & BRADLEY       30
JOHNSON ND WELD        21
CALINTON & KAINE       21
CASTLE AND BRADLEY     21
NAN                     5
Name: count, dtype: int64

In [20]:
# Cleaning Candidates

# Turning all general data to uppercase
general_data["candidate"] = general_data["candidate"].astype(str).str.upper()


unwanted_keywords = [
    r"\w*VOTE\w*",
    r"\w*UNCOM\w*",
    r"\w*TOTAL\w*",
    r"NAN",
    r"UNCERTIFIED",
    r"UNVERIFIED",
    r"NONE OF THE ABOVE",
    r"LBT",
    r"DEM",
    r"REP",
    r"GRN",
    r"NAN",
    r"\w*CANDIDATES\w*",
    "SCATTERING"
]

# Fixing FUENTE
general_data.loc[
    general_data["candidate"].str.contains("\w*FUENTE\w*", case=False, na=False),
    "candidate"
] = "FUENTE"
pattern = "|".join(unwanted_keywords)

# FILTER OUT TRASH WORDS
general_data = general_data[~general_data["candidate"].str.upper().str.contains(pattern, regex=True, na=False)]

# # Fixing Darrell Castle"
general_data.loc[
    general_data["candidate"].str.contains("\w*BRADLEY\w*", case=False, na=False),
    "candidate"
] = "CASTLE"

general_data.loc[
    general_data["candidate"].str.contains("\w*ATUREN\w*", case=False, na=False),
    "candidate"
] = "MATUREN"

general_data.loc[
    general_data["candidate"].str.contains("\w*OLTYSIK\w*", case=False, na=False),
    "candidate"
] = "SOLTYSIK"

general_data.loc[
    general_data["candidate"].str.contains("\w*WELD\w*", case=False, na=False),
    "candidate"
] = "JOHNSON"

general_data.loc[
    general_data["candidate"].str.contains("\w*MALDONADO\w*", case=False, na=False),
    "candidate"
] = "MALDONADO"

# # Fixing mixed president + vice_president by /
# general_data["candidate"] = (
#     general_data["candidate"].str.split("/")
#     .str[0]
#     .str.strip()
#     )

# #Selecting only last name
# general_data["candidate"] = general_data["candidate"].str.split().str[-1]

# Fixing the & candidate
general_data["candidate"] = (
    general_data["candidate"].str.split("&")
    .str[0]
    .str.strip()
    )


# # Fixing Cubbler
# general_data.loc[
#     ( 
#         general_data["candidate"].str.contains("\w*UBBIER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*UBLER\w*", case=False, na=False)|
#         general_data["candidate"].str.contains("\w*OBBLER\w*", case=False, na=False),
#     "candidate")
# ] = "CUBBLER"


# Fixing Kotlikoff
general_data.loc[
    general_data["candidate"].str.contains("\w*LIKOFF\w*", case=False, na=False),
    "candidate"
] = "KOTLIKOFF"

# Fixing Valdivia
general_data.loc[
    general_data["candidate"].str.contains("\w*KENISTON\W*", case=False, na=False),
    "candidate"
] = "KENISTON"

# Fixing HOEFLING
general_data.loc[
    general_data["candidate"].str.contains("\w*HOEFFLING", case=False, na=False),
    "candidate"
] = "HOEFLING"

general_data.loc[
    general_data["candidate"].str.contains("\w*SCHOENKE", case=False, na=False),
    "candidate"
] = "SCHOENKE"


# Fixing McMullin
general_data.loc[
    general_data["candidate"].str.contains("MCMULLIN", case=False, na=False) ,
    "candidate"
] = "MCMULLIN"

general_data.loc[
    general_data["candidate"].str.contains("FOX", case=False, na=False) ,
    "candidate"
] = "FOX"


general_data.loc[
    general_data["candidate"].str.contains("CALINTON", case=False, na=False) ,
    "candidate"
] = "CLINTON"

# Fixing LASTNAME + First name Initial
# general_data.loc[:,"candidate"] = (
#     general_data["candidate"].str.split().str[0].str.upper()
# )


# #Selecting only last name
general_data["candidate"] = general_data["candidate"].str.split().str[-1]

general_data["candidate"].value_counts(dropna=False)

candidate
TRUMP      767
JOHNSON    767
CLINTON    767
CASTLE     767
Name: count, dtype: int64

In [21]:
# Viewing Party
general_data["party"].value_counts(dropna=False)

party
REP    767
DEM    767
CON    767
LIB    695
LBT     72
Name: count, dtype: int64

In [22]:
#=====================================
# This function is used to look up party
# for the candidate in the same df
#=====================================
def fill_party_from_data(row, df):
    if pd.notna(row["party"]):
        return row["party"]
    
    # Try to find other rows with the same candidate and known party
    matches = df[(df["candidate"] == row["candidate"]) & (df["party"].notna())]
    if not matches.empty:
        return matches["party"].iloc[0]  # Return the first match's party
    else:
        return None  # Still unknown
    

#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Function to update the master candidate-party CSV
#=====================================
def update_master_candidate_party(df, master_path):
    # Read the existing master file
    master_df = pd.read_csv(master_path)

    # Filter and combine only known party entries (exclude 'UNK')
    new_data = (
        df[["candidate", "party"]]
        .dropna()
        .query('party != "UNK"')
        .drop_duplicates()
    )

    # Merge with master and remove duplicates by candidate
    updated_master = pd.concat([master_df, new_data]).drop_duplicates(subset="candidate")

    # Save updated version
    updated_master.to_csv(master_path, index=False)


#=====================================
# Example usage (uncomment and modify for your workflow)
#=====================================
# Fill party using internal data
# general_data["party"] = general_data.apply(
#     lambda row: fill_party_from_data(row, general_data),
#     axis=1
# )

# # Fill remaining party using general master CSV
# master_party_df = pd.read_csv(r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv") # USE YOUR OWN ADDRESS
# general_data = fill_party_from_master(general_data, master_party_df)
general_data["party"].value_counts(dropna=False)


party
REP    767
DEM    767
CON    767
LIB    695
LBT     72
Name: count, dtype: int64

In [23]:
# Cleaning Party
# Turning all general data party to uppercase
general_data["party"] = general_data["party"].astype(str).str.upper()

general_data["party"] = (
    general_data["party"]
    .replace({
        "REPUBLICAN": "REP",
        "R": "REP",
        "DEMOCRATIC": "DEM",
        "DEMOCRAT":"DEM",
        "DEM": "DEM",
        "D": "DEM",
        "GREEN": "GRN",
        "G": "GRN",
        "WGR":"GRN",
        "GREEN AND RAINBOW":"GRN",
        "GREEN-RAINBOW":"GRN",
        "LIBERTARIAN": "LIB",
        "LIBERTARIN":"LIB",
        "LPN":"LIB",
        "LBT": "LIB",
        "L": "LIB",
        "IAP":"CON",
        "CONSTITUTION": "CON",
        "AMERICAN DELTA":"AMD",
        "PROHIBITION":"PRO",
        "NP": "IND",
        "NON": "IND",
        "WRI": "IND",
        "WRITE-IN": "IND",
        "ONA": "IND",
        "GEN": "IND",
        "NONE":"UNK",
        "NPA":"IND",
        "UST":"CON",
        "NPP":"IND",
        "(WRITE-IN)":"IND"
    })
)

general_data["party"].value_counts(dropna=False)

party
REP    767
LIB    767
DEM    767
CON    767
Name: count, dtype: int64

In [24]:
# UPDATE MASTER FILE, CAREFUL
update_master_candidate_party(general_data, r"C:\Huy Phan\College\VoterTurnout\data\cleaned_data\gen_can_party.csv")

In [25]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_con_CASTLE,gen_dem_CLINTON,gen_lib_JOHNSON,gen_rep_TRUMP
0,AURORA1,4.0,62.0,11.0,230.0
1,AURORA2,3.0,23.0,7.0,120.0
2,AURORA3,6.0,101.0,19.0,290.0
3,AURORA5,1.0,61.0,11.0,128.0
4,AURORA7,5.0,93.0,26.0,206.0
...,...,...,...,...,...
762,ZIEBACHPRECINCT-3 DAVIS,5.0,99.0,12.0,67.0
763,ZIEBACHPRECINCT-4 GLAD VALLEY,0.0,10.0,0.0,59.0
764,ZIEBACHPRECINCT-6 REDELM (PRECINCT-5 NORTH DUP...,10.0,112.0,14.0,215.0
765,ZIEBACHPRECINCT-7 RED SCAFFOLD,3.0,36.0,1.0,13.0


In [26]:
# Merge
combined = pd.merge(primary_result, general_result, on="precinct", how="inner")

# Convert DEM primary columns to numeric and calculate total
dem_cols = combined.filter(like="pri_dem_").columns
combined[dem_cols] = combined[dem_cols].apply(pd.to_numeric, errors="coerce")
combined["dem_primary_total"] = combined[dem_cols].sum(axis=1)

# Convert general election columns to numeric and calculate total
gen_cols = combined.filter(like="gen_").columns
combined[gen_cols] = combined[gen_cols].apply(pd.to_numeric, errors="coerce")
combined["general_total"] = combined[gen_cols].sum(axis=1)

# Convert all numeric columns to int
for col in combined.columns[1:]:
    combined[col] = pd.to_numeric(combined[col], errors='coerce').fillna(0).astype(int)

# Identify filtered-out precincts
primary_precincts = set(primary_result["precinct"])
general_precincts = set(general_result["precinct"])
combined_precincts = set(combined["precinct"])

primary_filtered_out = primary_precincts - combined_precincts
general_filtered_out = general_precincts - combined_precincts




In [27]:


combined.to_csv("SD.csv", index=False)
# Step 3: Identify unmatched precincts and retrieve full rows
combined_precincts = set(combined["precinct"])

# Rows in primary_result but not in combined
primary_filtered_out = primary_result[~primary_result["precinct"].isin(combined_precincts)]
primary_filtered_out.to_csv("SD_primary_filtered.csv", index=False)

# Rows in general_result but not in combined
general_filtered_out = general_result[~general_result["precinct"].isin(combined_precincts)]
general_filtered_out.to_csv("SD_general_filtered.csv", index=False)

In [28]:
print(f"primary: {len(primary_precincts)}, general: {len(general_precincts)}, combined: {len(combined)}")

primary: 30, general: 767, combined: 26


In [30]:
print(f"primary: {len(pri_combined_df['county'].unique())}, general: {len(gen_combined_df['county'].unique())}")


primary: 4, general: 66


In [ ]:
pri_combined_df['jurisdiction'].unique()

array(['Carson City', 'Churchill', 'Clark', 'Douglas', 'Elko',
       'Esmeralda', 'Eureka', 'Humboldt', 'Lander', 'Lincoln', 'Lyon',
       'Mineral', 'Nye', 'ye', 'Pershing', 'Storey', 'Washoe',
       'White Pine'], dtype=object)

In [ ]:
gen_combined_df['county'].unique()

array(['CARSON CITY', 'CHURCHILL', 'CLARK', 'DOUGLAS', 'ELKO',
       'ESMERALDA', 'EUREKA', 'HUMBOLDT', 'LANDER', 'LINCOLN', 'LYON',
       'MINERAL', 'NYE', 'PERSHING', 'STOREY', 'WASHOE', 'WHITE PINE'],
      dtype=object)

In [ ]:
primary_counties = set(pri_combined_df['jurisdiction'].dropna().str.strip().str.upper())
general_counties = set(gen_combined_df['county'].dropna().str.strip().str.upper())

diff = primary_counties - general_counties
print(f"Counties in primary but not in general: {len(diff)}")
print(diff)


Counties in primary but not in general: 1
{'YE'}
